<a href="https://colab.research.google.com/github/AndyFreeeeman/Model_Predict/blob/main/Hyperparameter_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)

In [3]:
# import dataset

from sklearn.datasets import load_wine

wine = load_wine()
# 輸入特徵
X = wine.data
x = wine.data
# 輸出特徵
y = wine.target

print(X)
print(y)

[[  14.23    1.71    2.43 ...    1.04    3.92 1065.  ]
 [  13.2     1.78    2.14 ...    1.05    3.4  1050.  ]
 [  13.16    2.36    2.67 ...    1.03    3.17 1185.  ]
 ...
 [  13.27    4.28    2.26 ...    0.59    1.56  835.  ]
 [  13.17    2.59    2.37 ...    0.6     1.62  840.  ]
 [  14.13    4.1     2.74 ...    0.61    1.6   560.  ]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [4]:
# 輸入資料

wine = load_wine()
df_data = pd.DataFrame(data= np.c_[wine['data'], wine['target']],
                     columns= ['alcohol','malic_acid','ash','alcalinity_of_ash','magnesium','total_phenols','flavanoids','nonflavanoid_phenols','proanthocyanins','color_intensity','hue','od280/od315_of_diluted_wines','proline','class'])
df_data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2.0


In [5]:
# 切割訓練集與測試集

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print('train shape:', X_train.shape)
print('test shape:', X_test.shape)

train shape: (124, 13)
test shape: (54, 13)


In [6]:
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def objective(trial, X=X, y=y):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna.
    """
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000, 100),
        'eta': trial.suggest_float("eta", 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    reg = xgb.XGBRegressor(**params)
    reg.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)], eval_metric='rmse',
            verbose=False)
    return mean_squared_error(y_valid, reg.predict(X_valid), squared=False)

In [20]:
%%time
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 200)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2022-09-07 09:14:37,872] A new study created in memory with name: no-name-5af21ef8-6914-483b-b6a1-44eb3cfbbfb4


[09:14:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-09-07 09:14:38,143] Trial 0 finished with value: 0.22810602947650108 and parameters: {'max_depth': 11, 'subsample': 0.3398705756290469, 'n_estimators': 600, 'eta': 6.577521512236864e-05, 'alpha': 7.652928370594759e-06, 'lambda': 0.00045154489828277505, 'gamma': 0.001324652037757677, 'min_child_weight': 9, 'grow_policy': 'lossguide', 'colsample_bytree': 0.5132104384832601}. Best is trial 0 with value: 0.22810602947650108.


[09:14:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-09-07 09:14:38,996] Trial 1 finished with value: 0.26848779337325074 and parameters: {'max_depth': 8, 'subsample': 0.9699641944937203, 'n_estimators': 1400, 'eta': 9.400130736421884e-08, 'alpha': 3.3280637328786923e-07, 'lambda': 0.0006812762333753705, 'gamma': 0.0001631049122385358, 'min_child_weight': 3, 'grow_policy': 'depthwise', 'colsample_bytree': 0.7794911018085842}. Best is trial 0 with value: 0.22810602947650108.


...

[09:17:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-09-07 09:17:14,692] Trial 198 finished with value: 0.20857697463280434 and parameters: {'max_depth': 14, 'subsample': 0.6770188563522019, 'n_estimators': 1100, 'eta': 0.0021629481673801545, 'alpha': 5.448125840347096e-05, 'lambda': 3.851419292979382e-06, 'gamma': 0.00016400592654838684, 'min_child_weight': 3, 'grow_policy': 'lossguide', 'colsample_bytree': 0.9130872383634868}. Best is trial 159 with value: 0.11962052819030741.


[09:17:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-09-07 09:17:15,459] Trial 199 finished with value: 0.15266428816086094 and parameters: {'max_depth': 14, 'subsample': 0.9357907821662103, 'n_estimators': 1200, 'eta': 3.058372370715306e-05, 'alpha': 9.119037662885004e-05, 'lambda': 2.2530117141886196e-06, 'gamma': 0.00012045413297113979, 'min_child_weight': 6, 'grow_policy': 'lossguide', 'colsample_bytree': 0.9480906356685618}. Best is trial 159 with value: 0.11962052819030741.


Number of finished trials: 200
Best trial parameters: {'max_depth': 12, 'subsample': 0.915611586602131, 'n_estimators': 1200, 'eta': 3.229557396077413e-08, 'alpha': 3.75085691197433e-08, 'lambda': 1.1220293133959129e-05, 'gamma': 8.628162610021033e-05, 'min_child_weight': 3, 'grow_policy': 'lossguide', 'colsample_bytree': 0.9270066219577807}
Best score: 0.11962052819030741
CPU times: user 2min 33s, sys: 4.39 s, total: 2min 38s
Wall time: 2min 37s


In [12]:
from optuna.samplers import CmaEsSampler, RandomSampler

# Study with a random sampler
study = optuna.create_study(sampler=RandomSampler(seed=42))

# Study with a CMA ES sampler
study = optuna.create_study(sampler=CmaEsSampler(seed=42))

[I 2022-09-07 09:10:33,627] A new study created in memory with name: no-name-790f949e-9566-48bc-a195-1a44ea882773
[I 2022-09-07 09:10:33,629] A new study created in memory with name: no-name-dcd0a037-6c16-4517-bf56-e7087c923dbb


In [21]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

In [22]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)